## Q learning demo of Frozen Lake Game Solution
Ou Deng on Nov 19,2022  
ref: https://space.bilibili.com/1119972180/channel/seriesdetail?sid=2515026

### What's [Q-learning](https://en.wikipedia.org/wiki/Q-learning){:target="_blank"}

tips: commend+/: rem line(s) in Jupyter

- GYM
- Bellman equation


### 1.GYM

https://www.gymlibrary.dev/  
https://www.gymlibrary.dev/environments/toy_text/frozen_lake/  


In [1]:
#!pip install gym

     |████████████████████████████████| 721 kB 13.3 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827633 sha256=bd4d4999eedc8918a7809c6de06ca600b7109f458edccb88b9c017b3085526ac
  Stored in directory: /Users/deng/Library/Caches/pip/wheels/af/2b/30/5e78b8b9599f2a2286a582b8da80594f654bf0e18d825a4405
Successfully built gym


Installation message of ```pip install gym```
> Successfully built gym  
Installing collected packages: gym-notices, gym  
Successfully installed gym-0.26.2 gym-notices-0.0.8

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [276]:
import gym
desc=["SFFF", "FHFH", "FFFH", "HFFG"]
env = gym.make('FrozenLake-v1', desc=desc, map_name="4x4", is_slippery=True)
state = env.reset()

In [277]:
type(env)

gym.wrappers.time_limit.TimeLimit

In [281]:
state

(0, {'prob': 1})

In [278]:
type(state)

tuple

In [279]:
X = list(state)
# Tuple is uneditable.
# Transfer to list and only take the no.0 value which is the state number in env initiao setting.

In [280]:
X[0]

0

In [61]:
# Visualize env (not successful for env.render())


In [66]:
# Action on the FrozenLake
# 0: LEFT; 1: DOWN; 2: RIGHT; 3: UP
env.reset()
env.step(2) 

(0, 0.0, False, False, {'prob': 0.3333333333333333})

## Bellman equation

$$Q(S_t, a_t) \leftarrow Q(S_t, a_t)+\alpha [r_{t+1}+\lambda \sideset{}{}{max}_a (S_t,a)-Q(S_t, a_t))]$$

\begin{align}
  Q:&\text{quality, Q-table} \\
  S:&\text{state}\\
  a:&\text{action}\\
  \alpha:&\text{learning rate} \\
  r:&\text{reward} \\
  \lambda:&\text{discount, for blancing the short-term reward and long-term reward.}\\
\end{align}

More detailed info: https://towardsdatascience.com/the-bellman-equation-59258a0d3fa7

### 4x4 map (Perfect)

In [250]:
import gym
desc=["SFFF", "FHFH", "FFFH", "HFFG"] #S:start; F:Frozen surface; H:Hole; G:Goal.
env = gym.make('FrozenLake-v1', desc=desc, map_name="4x4", is_slippery=True)
state = env.reset()

import numpy as np
q_table = np.zeros((16,4)) #Initialize the Q-table.

### 8x8 map (Not bad)

In [475]:
import gym
desc=["SFFFFFFF","FFFFFFFF","FFFHFFFF","FFFFFHFF","FFFHFFFF","FHHFFFHF","FHFFHFHF","FFFHFFFG",]
env = gym.make('FrozenLake-v1', desc=desc, map_name="8x8", is_slippery=True)
state = env.reset()

import numpy as np
q_table = np.zeros((64,4)) #Initialize the Q-table.

import random
random.seed(2022)

N = 2000
alpha = 0.5
lam = 0.9
e1 = 0.99
e2 = 0.9995

### 5x5 map (Good)

In [1]:
import gym
desc=[
    "SFHFF",
    "FFFFH",
    "HFFFF",
    "FFHFG"] #S:start; F:Frozen surface; H:Hole; G:Goal.
env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=True)
state = env.reset()

import numpy as np
q_table = np.zeros((25,4)) #Initialize the Q-table.

import random
random.seed(2022)

N = 1000
alpha = 0.5
lam = 0.9
e1 = 0.99
e2 = 0.999

In [2]:

train_times=10 # If train more times, the Q-table might be add to the bigger value causing bed result.
for k in range(train_times):
    for i in range(N):
        done = False
        state0 = env.reset()
        state = list(state0)[0]
        e1 *= e2
        #e1 -= 0.0001
        while not done:
            if random.random() < e1 or np.sum(q_table) == 0:
                action = random.randint(0,3) #move direction: Left-Down-Right-Up
            else:
                action = np.argmax(q_table[state,:])
                
            new_state, reward, safe, done, info = env.step(action)

            # Bellman equation
            q_table[state, action] = q_table[state, action] + \
            alpha * (reward + lam * np.max(q_table[new_state,:]) - q_table[state, action])

            state = new_state
            # if q_table[state, action] >0:
            #    print(state, action, q_table[state, action])

    # Evaluate 
    success_times = 0
    for j in range(N):
        done = False
        state = env.reset()
        state = list(state)[0]
        while not done:
            action = np.argmax(q_table[state,:])
            new_state, reward, safe, done, info = env.step(action)    
            state = new_state
            if reward == 1:
                success_times += 1
    print("Train times, #",k+1, ":the Success ratio of S to G:",\
          success_times,"/", N, "=", '{:.1f}%'.format(success_times / N * 100))
    
    print("e1=",e1,";","np.sum(q_table)=",np.sum(q_table))

q_table # as no hole on the fronzen lake.

Train times, # 1 :the Success ratio of S to G: 589 / 1000 = 58.9%
e1= 0.36401847052325514 ; np.sum(q_table)= 9.042801265901277
Train times, # 2 :the Success ratio of S to G: 465 / 1000 = 46.5%
e1= 0.1338479261435251 ; np.sum(q_table)= 7.777293503436477
Train times, # 3 :the Success ratio of S to G: 131 / 1000 = 13.1%
e1= 0.04921527005805615 ; np.sum(q_table)= 6.970125349419392
Train times, # 4 :the Success ratio of S to G: 859 / 1000 = 85.9%
e1= 0.018096229629214665 ; np.sum(q_table)= 6.579618289274168
Train times, # 5 :the Success ratio of S to G: 866 / 1000 = 86.6%
e1= 0.006653900840266982 ; np.sum(q_table)= 6.0586509422995976
Train times, # 6 :the Success ratio of S to G: 853 / 1000 = 85.3%
e1= 0.0024466088958458427 ; np.sum(q_table)= 5.838969928199119
Train times, # 7 :the Success ratio of S to G: 869 / 1000 = 86.9%
e1= 0.0008996068972064574 ; np.sum(q_table)= 6.235407413164911
Train times, # 8 :the Success ratio of S to G: 861 / 1000 = 86.1%
e1= 0.000330781340195217 ; np.sum(q_tab

array([[1.99825901e-02, 6.11784677e-02, 2.04316137e-02, 2.05599276e-02],
       [1.07724872e-01, 8.20219602e-03, 3.33733570e-03, 9.61150664e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.98608292e-03, 1.04375560e-02, 1.19561765e-01, 6.15186997e-03],
       [8.39507679e-03, 7.06450429e-03, 4.82442825e-03, 8.86413713e-02],
       [1.14620318e-03, 1.10282106e-02, 5.21967587e-03, 7.15751520e-02],
       [2.67216862e-02, 1.77482758e-02, 1.50691942e-01, 2.71848310e-02],
       [1.69777138e-02, 1.95565033e-01, 4.16186833e-02, 4.00623393e-02],
       [2.23437918e-01, 1.36667497e-02, 2.51773888e-02, 2.24540136e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.07766256e-02, 1.81741828e-02, 1.76213297e-01, 5.19596435e-03],
       [1.49286476e-02, 5.31560393e-02, 4.21280339e-03, 3.86604034e-01],
       [1.15473267e-01, 1.17950128e-01, 4.07922929e

In [3]:
new_state, reward, safe, done, info

(19, 0, True, True, {'prob': 1.0})

### Result memo

73.3% after three times training in 4x4 map under the following conditions:  
N = 1000  
alpha = 0.9  
lam = 0.9  
e1 = 0.2  
e2 = 0.99  

Start at 20% random move and the rest follows Q-table.

However, I failed on 8x8 map training even on the successful hyper-parameters of 4x4 map. Uncertain reason.  
Back to try from 5x5 map.

desc=[
    "SFHFF",
    "FFFFH",
    "HFFFF",
    "FFHFG"]
    
N = 10000 # 10k train times  
alpha = 0.5  
lam = 0.8  
e1 = 0.9  
e2 = 0.99  

Bad(most 0% successful ratio.)

Then change e1 to 0.99, and e2 to 0.999. More randomly trails.
Works. Best result to 88%. Raise fastly at the beginning. 

However, few round not stable engough.  
Abd some lines, especially the G horizon line, semms challenging to train.

The random moves in the first 1000 train times sound good. Otherwise, the results tend to get worse. e.g.: e1 = 0.9 and e2 = 0.9999.

Now, we use 5x5 map parameter setting to run 8x8 map. Terrible, the 0% successful ratio.  


### Improve result on 8x8 maps

Train 10 times under the following parameters:  
N = 2000  
alpha = 0.5  
lam = 0.9  
e1 = 0.99  
e2 = 0.9995  

The result is between **40%-50%**, the details as follows:  


Train times, # 1 :the Success ratio of S to G: 861 / 2000 = 43.0%  
e1= 0.3641095776245519 ; np.sum(q_table)= 10.221157433044446  
Train times, # 2 :the Success ratio of S to G: 211 / 2000 = 10.5%  
e1= 0.13391493385649383 ; np.sum(q_table)= 5.8994332121056345  
Train times, # 3 :the Success ratio of S to G: 649 / 2000 = 32.5%  
e1= 0.049252232327381594 ; np.sum(q_table)= 6.44240765023446  
Train times, # 4 :the Success ratio of S to G: 0 / 2000 = 0.0%  
e1= 0.018114353040191024 ; np.sum(q_table)= 2.993239018652109  
Train times, # 5 :the Success ratio of S to G: 204 / 2000 = 10.2%  
e1= 0.00666223175192522 ; np.sum(q_table)= 4.130704359368514  
Train times, # 6 :the Success ratio of S to G: 1070 / 2000 = 53.5%  
e1= 0.0024502852416468253 ; np.sum(q_table)= 4.194490041588694  
Train times, # 7 :the Success ratio of S to G: 953 / 2000 = 47.6%  
e1= 0.0009011841660562588 ; np.sum(q_table)= 3.804560389424127  
Train times, # 8 :the Success ratio of S to G: 1086 / 2000 = 54.3%  
e1= 0.000331444228348159 ; np.sum(q_table)= 3.906513800103143  
Train times, # 9 :the Success ratio of S to G: 1047 / 2000 = 52.3%  
e1= 0.00012190102827267 ; np.sum(q_table)= 2.214332314198644  
Train times, # 10 :the Success ratio of S to G: 853 / 2000 = 42.6%  
e1= 4.483366860238418e-05 ; np.sum(q_table)= 2.720206247364254  

array([[5.50943857e-04, 3.77459788e-03, 5.48095134e-04, 7.57922637e-04],
       [7.66037350e-04, 6.91727817e-04, 4.37279478e-03, 7.17143622e-04],
       [9.35400237e-04, 5.37712890e-03, 9.94436218e-04, 1.00571765e-03],
       [1.44455997e-03, 1.31561907e-02, 1.50347912e-03, 1.50894275e-03],
       [2.13821894e-03, 1.45206912e-02, 2.23034318e-03, 2.27870942e-03],
       [3.23885865e-03, 3.47511889e-03, 1.64381522e-02, 3.15667511e-03],
       [2.24293690e-02, 4.28615438e-03, 4.29799317e-03, 4.01387118e-03],
       [2.01889312e-02, 4.64920040e-03, 4.70335469e-03, 4.66193601e-03],
       [5.52267881e-04, 5.55819844e-04, 3.08258115e-03, 5.63964394e-04],
       [6.63187784e-04, 6.26676950e-04, 6.61785265e-03, 6.53684555e-04],
       [8.87225727e-04, 8.72013140e-04, 8.20403031e-04, 5.87540019e-03],
       [1.06172137e-03, 9.71677256e-04, 1.12855966e-03, 7.50141700e-03],
       [2.01363655e-03, 1.28910587e-02, 2.25483148e-03, 2.07692365e-03],
       [3.53777030e-03, 3.56485917e-03, 2.51001309e-02, 3.09058145e-03],
       [4.58312760e-03, 4.38486406e-03, 3.32087391e-02, 4.30840731e-03],
       [5.22719864e-03, 5.28025256e-03, 3.90304872e-02, 5.16504253e-03],
       [5.20976007e-04, 5.37608148e-04, 5.36691051e-04, 2.88166545e-03],
       [5.17739883e-04, 5.53601445e-04, 5.35503516e-04, 4.17491483e-03],
       [4.87338107e-03, 1.84402687e-04, 4.79027696e-04, 3.20065003e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.83640493e-04, 6.69973129e-04, 1.51973352e-02, 1.32581946e-03],
       [2.70994342e-03, 2.58117823e-03, 2.34509053e-03, 3.66862473e-02],
       [5.11494904e-03, 5.74187255e-03, 6.61244632e-02, 5.61879375e-03],
       [7.01686866e-03, 5.38242035e-02, 7.00298490e-03, 7.07799664e-03],
       [3.61044016e-04, 3.59477618e-04, 1.60060206e-03, 3.21554573e-04],
       [3.53595329e-04, 4.12681048e-04, 3.54795680e-04, 3.42175835e-03],
       [2.35557205e-04, 2.54359097e-04, 1.62804290e-04, 1.53234103e-03],
       [2.17275587e-05, 2.50927447e-05, 4.50060455e-05, 1.55473276e-03],
       [1.88366454e-03, 6.88242512e-05, 2.81054740e-04, 2.14783086e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.09353792e-03, 4.71622112e-03, 1.06956068e-01, 3.58489310e-03],
       [1.04678084e-02, 1.03260268e-02, 8.65931165e-02, 9.27151845e-03],
       [2.09940107e-04, 2.31105810e-04, 3.65349498e-04, 6.10957728e-04],
       [9.18662572e-05, 7.89943719e-05, 1.04785202e-04, 6.06704292e-04],
       [1.52040264e-03, 2.90876870e-05, 5.23079645e-06, 5.71080952e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.98908096e-04, 3.62419276e-04, 5.47539385e-03, 1.45235310e-04],
       [5.84760522e-04, 1.74659664e-02, 5.88866066e-04, 2.66514121e-04],
       [1.22999757e-03, 2.09518271e-03, 1.32944465e-03, 1.33154319e-01],
       [1.90935527e-02, 1.28826688e-02, 1.04043226e-01, 1.74754979e-02],
       [1.21120228e-03, 1.34477783e-04, 1.19785791e-04, 2.20002321e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.69824686e-06, 1.45424353e-04, 1.83726986e-06, 1.65366233e-06],
       [1.06495551e-04, 8.67665976e-05, 1.38515914e-04, 3.09110183e-03],
       [4.38501834e-03, 1.75420965e-04, 1.30764285e-04, 1.39289655e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.81632482e-02, 3.05732099e-02, 4.10174211e-01, 3.42583645e-02],
       [5.51883620e-04, 6.42132211e-05, 7.97660349e-05, 8.73733721e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.57372671e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.47066684e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.88119234e-06, 2.96678053e-06, 4.55405035e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.17329824e-02, 1.08172437e-01, 7.93020092e-01, 5.80382519e-02],
       [7.77985817e-05, 7.94751325e-05, 7.79131341e-05, 3.17606314e-04],
       [2.31798596e-05, 2.00970007e-04, 2.41618448e-05, 0.00000000e+00],
       [6.26418049e-05, 0.00000000e+00, 0.00000000e+00, 5.37827159e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.71977395e-04, 6.27392590e-07, 0.00000000e+00, 9.85840965e-07],
       [3.61151794e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])


## Be exhausted on hyper-parameter trails. 8x8 map results stop at about 50% success ratio.